In [25]:
pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
with open("Shakespeare.txt","r") as f:
    text = f.read()


In [27]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [28]:
#get unique characters in the dataset in a sorted order
#possible characters model can see
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size)

65


In [29]:
#character level tokenizer: produces id for all characters
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
#create anonymous function to map characters to integer value
encode = lambda s: [stoi[c] for c in s]
#create anonymous function to map integers to character
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Hello my name is"))
print(decode(encode("Hello my name is")))


[20, 43, 50, 50, 53, 1, 51, 63, 1, 52, 39, 51, 43, 1, 47, 57]
Hello my name is


In [30]:

import torch

data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape, data.dtype)
print(data[:200])


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59])


In [31]:
#create validation and training sets

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

when training, you train the model on blocks(amount of inferences you want to train on)

for block size n: take n+1 samples of data



In [32]:
block_size = 8
print(train_data[:block_size + 1])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


want to use context varying up to size of block, allows prediction for inputs up to block size
then truncates greater than block size
prediction given context: [1,2,3,4,5] and targets: [2,3,4,5,6]
context [1,2] has target [3] (correspondign to last i in target)

In [33]:
x = train_data[:block_size]
y = train_data[1:block_size +1]
for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print(f"context:{context} for target:{target}")
    

context:tensor([18]) for target:47
context:tensor([18, 47]) for target:56
context:tensor([18, 47, 56]) for target:57
context:tensor([18, 47, 56, 57]) for target:58
context:tensor([18, 47, 56, 57, 58]) for target:1
context:tensor([18, 47, 56, 57, 58,  1]) for target:15
context:tensor([18, 47, 56, 57, 58,  1, 15]) for target:47
context:tensor([18, 47, 56, 57, 58,  1, 15, 47]) for target:58


batches: multiple chunks of context processed independently for efficiency

In [34]:
torch.manual_seed(1337)

batch_size = 4
block_size = 8


def get_batch(split):
    data = train_data if split=="train" else val_data
    #generate 1D tensor of length batch_size
    ix = torch.randint(len(data) - block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size]for i in ix])
    y = torch.stack([data[i+1:i+block_size+1]for i in ix])
    return x,y

train_x,train_y = get_batch("train")
print(train_x.shape)
print(train_x)

print(train_y.shape)
print(train_y)



torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
